# Libraries

In [3]:
import numpy as np
import pandas as pd
import sympy as sym
pi = np.pi

# Ideal projectors

In [4]:
def proj(j,n):
#     mat = np.zeros((3,3))
    temp1 = np.cos(pi/n)/(1+np.cos(pi/n))
    temp2 = 1-temp1
    temp3 = j*pi*(n-1)/n
    vec = np.asarray([np.sqrt(temp1),np.sqrt(temp2)*np.sin(temp3), np.sqrt(temp2)*np.cos(temp3)])
    mat = np.outer(vec,vec)
    return mat    

In [3]:
# Orthogonality check
i = 0
np.dot(proj((i+1) % 5,5),proj(i,5))

array([[ 8.82680578e-18,  0.00000000e+00, -1.28106935e-17],
       [ 1.03962966e-17,  0.00000000e+00,  2.53568037e-18],
       [ 2.80201435e-18,  0.00000000e+00,  9.69177354e-19]])

In [4]:
#Projection check
i = 0
proj(i,5)@proj(i,5) - proj(i,5) <= 1.0e-10

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

# Translation operator

In [5]:
T = np.matrix([[0,0,1],
               [1,0,0],
               [0,1,0]])

In [6]:
np.transpose(T)@T

matrix([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])

# Cycles

## 5 cycle

In [7]:
P1 = proj(1,5)
P2 = proj(2,5)
P3 = proj(3,5)
P4 = proj(4,5)
P5 = proj(5,5)

In [8]:
P1

array([[ 0.4472136 ,  0.29225023, -0.40224793],
       [ 0.29225023,  0.19098301, -0.26286556],
       [-0.40224793, -0.26286556,  0.3618034 ]])

In [9]:
x1 = sym.Symbol('x1')
x2 = sym.Symbol('x2')
x3 = sym.Symbol('x3')
x4 = sym.Symbol('x4')
x5 = sym.Symbol('x5')
x6 = sym.Symbol('x6')
x7 = sym.Symbol('x7')
x8 = sym.Symbol('x8')
x9 = sym.Symbol('x9')
x10 = sym.Symbol('x10')

In [10]:
LHS = x1*P1 + x2*P2 + x3*P3 + x4*P4 + x5*P5 + x6*(P5@P3) + x7*(P5@P2) + x8*(P2@P4) + x9*(P3@P1)
#LHS = x1*P1 + x2*P2 + x3*P3 + x4*P4 + x5*P5 + x6*(P1@P3) + x7*(P2@P4) + x8*(P3@P5) + x9*(P4@P1) +x10*(P5@P2)

In [11]:
LHSP = pd.DataFrame(LHS)

In [12]:
LHSP

,0,1,2
0,0.447213595499958*x1 + 0.447213595499958*x2 + ...,0.292250229469488*x1 - 0.472870804501588*x2 + ...,-0.402247932095355*x1 + 0.153645038156066*x2 +...
1,0.292250229469488*x1 - 0.472870804501588*x2 + ...,0.190983005625053*x1 + 0.5*x2 + 0.5*x3 + 0.190...,-0.262865556059567*x1 - 0.162459848116453*x2 +...
2,-0.402247932095355*x1 + 0.153645038156066*x2 +...,-0.262865556059567*x1 - 0.162459848116453*x2 +...,0.361803398874989*x1 + 0.052786404500042*x2 + ...


In [13]:
Equations = LHS - T

In [14]:
Equations;

In [15]:
P1@P3 - P1@P4

array([[-5.55111512e-17,  4.72870805e-01,  3.43560750e-01],
       [-5.55111512e-17,  3.09016994e-01,  2.24513988e-01],
       [ 5.55111512e-17, -4.25325404e-01, -3.09016994e-01]])

In [16]:
eqset1 = [] #set of equations
for i in range(3):
    for j in range(3):
        eqset1.append(Equations[i,j])
solution5 = sym.solve((eqset1),(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10))

In [17]:
solution5

{x9: -1.45863889754039,
 x8: -6.16165530330893,
 x7: 4.62281353472131,
 x4: 2.28088533047804,
 x5: -3.40789916832931,
 x6: 2.99748066612802,
 x3: 2.06682268604037,
 x2: 0.536580618098216,
 x1: -1.47638946628733}

### Sanity check

In [18]:
#LHS = y1*P1 + y2*P2 - P1

In [19]:
y1 = sym.Symbol('y1')
y2 = sym.Symbol('y2')
eqset = [] #set of equations
for i in range(3):
    for j in range(3):
        eqset.append(LHS[i,j])
solution = sym.solve((eqset),(y1,y2))

In [20]:
solution

[]

### Isometry

In [21]:
# S = TUVU
# We need three basis vectors. Let these be P1, P2 and P3.
# First we construct the basis vectors
def basis(j,n):
    temp1 = np.cos(pi/n)/(1+np.cos(pi/n))
    temp2 = 1-temp1
    temp3 = j*pi*(n-1)/n
    vec = np.asarray([np.sqrt(temp1),np.sqrt(temp2)*np.sin(temp3), np.sqrt(temp2)*np.cos(temp3)])
    return vec      

In [22]:
b1 = basis(0,5)
b2 = basis(1,5)
b3 = np.cross(b1,b2)

In [23]:
# Sanity check
np.inner(b2,b3)

-2.7755575615628914e-17

In [24]:
coeff = [solution5[x1],solution5[x2],solution5[x3],solution5[x4],solution5[x5],solution5[x6],solution5[x7],solution5[x8],solution5[x9]]

In [25]:
coeff;

## 7 cycle

In [7]:
P1 = proj(1,7)
P2 = proj(2,7)
P3 = proj(3,7)
P4 = proj(4,7)
P5 = proj(5,7)
P6 = proj(6,7)
P7 = proj(7,7)

In [8]:
x1 = sym.Symbol('x1')
x2 = sym.Symbol('x2')
x3 = sym.Symbol('x3')
x4 = sym.Symbol('x4')
x5 = sym.Symbol('x5')
x6 = sym.Symbol('x6')
x7 = sym.Symbol('x7')
x8 = sym.Symbol('x8')
x9 = sym.Symbol('x9')
x10 = sym.Symbol('x10')

In [9]:
LHS = x1*P1 + x2*P2 + x3*P3 + x4*P4 + x5*P5 + x6*(P5@P3) + x7*(P5@P2) + x8*(P2@P4) + x9*(P3@P1)

In [10]:
Equations = LHS - T
eqset1 = [] #set of equations
for i in range(3):
    for j in range(3):
        eqset1.append(Equations[i,j])
solution7 = sym.solve((eqset1),(x1,x2,x3,x4,x5,x6,x7,x8,x9))


solution7

{x1: -2.48781484986017,
 x2: 0.174298227013938,
 x3: 0.419454546714465,
 x4: 1.75603533644891,
 x5: 1.45995537639807,
 x6: -2.09492067480099,
 x7: 1.64841804755391,
 x8: -3.16439013280000,
 x9: 2.87727771306281}

In [20]:
sol7 = sym.solve(sym.Matrix(LHS-T),[x1,x2,x3,x4,x5,x6,x7,x8,x9])

sol7

{x1: -2.48781484986017,
 x2: 0.174298227013938,
 x3: 0.419454546714465,
 x4: 1.75603533644891,
 x5: 1.45995537639807,
 x6: -2.09492067480099,
 x7: 1.64841804755391,
 x8: -3.16439013280000,
 x9: 2.87727771306281}

## 9 cycle

In [36]:
N = 9

P1 = proj(1,N)
P2 = proj(2,N)
P3 = proj(3,N)
P4 = proj(4,N)
P5 = proj(5,N)
P6 = proj(6,N)
P7 = proj(7,N)
P8 = proj(6,N)
P9 = proj(7,N)

In [37]:
x1 = sym.Symbol('x1')
x2 = sym.Symbol('x2')
x3 = sym.Symbol('x3')
x4 = sym.Symbol('x4')
x5 = sym.Symbol('x5')
x6 = sym.Symbol('x6')
x7 = sym.Symbol('x7')
x8 = sym.Symbol('x8')
x9 = sym.Symbol('x9')
x10 = sym.Symbol('x10')

In [38]:
LHS = x1*P1 + x2*P2 + x3*P3 + x4*P4 + x5*P5 + x6*(P5@P3) + x7*(P5@P2) + x8*(P2@P4) + x9*(P3@P1)

In [39]:
Equations = LHS - T
eqset1 = [] #set of equations
for i in range(3):
    for j in range(3):
        eqset1.append(Equations[i,j])
solution9 = sym.solve((eqset1),(x1,x2,x3,x4,x5,x6,x7,x8,x9))
solution9

{x1: -3.33356971686325,
 x2: 0.584484137492421,
 x3: -1.64695030531387,
 x4: 1.12694503027855,
 x5: 4.18017762031205,
 x6: -5.32945475470260,
 x7: 0.742724736560921,
 x8: -2.45848358961390,
 x9: 6.56043446521856}

## 11 cycle

In [40]:
N = 11

P1 = proj(1,N)
P2 = proj(2,N)
P3 = proj(3,N)
P4 = proj(4,N)
P5 = proj(5,N)
P6 = proj(6,N)
P7 = proj(7,N)
P8 = proj(6,N)
P9 = proj(7,N)

In [41]:
x1 = sym.Symbol('x1')
x2 = sym.Symbol('x2')
x3 = sym.Symbol('x3')
x4 = sym.Symbol('x4')
x5 = sym.Symbol('x5')
x6 = sym.Symbol('x6')
x7 = sym.Symbol('x7')
x8 = sym.Symbol('x8')
x9 = sym.Symbol('x9')
x10 = sym.Symbol('x10')

In [42]:
LHS = x1*P1 + x2*P2 + x3*P3 + x4*P4 + x5*P5 + x6*(P5@P3) + x7*(P5@P2) + x8*(P2@P4) + x9*(P3@P1)

In [43]:
Equations = LHS - T
eqset1 = [] #set of equations
for i in range(3):
    for j in range(3):
        eqset1.append(Equations[i,j])
solution11 = sym.solve((eqset1),(x1,x2,x3,x4,x5,x6,x7,x8,x9))
solution11

{x1: -4.35262378386501,
 x2: 1.11139587164724,
 x3: -3.69984367046470,
 x4: 0.550210499217815,
 x5: 6.66521986415699,
 x6: -8.57718949080699,
 x7: 0.187251806349016,
 x8: -2.15920232598671,
 x9: 10.4061693018324}

## General N-cycle?

In [55]:
N = 31

P1 = proj(1,N)
P2 = proj(2,N)
P3 = proj(3,N)
P4 = proj(4,N)
P5 = proj(5,N)
P6 = proj(6,N)
P7 = proj(7,N)
P8 = proj(6,N)
P9 = proj(7,N)

In [56]:
LHS = x1*P1 + x2*P2 + x3*P3 + x4*P4 + x5*P5 + x6*(P5@P3) + x7*(P5@P2) + x8*(P2@P4) + x9*(P3@P1)
Equations = LHS - T
eqset1 = [] #set of equations
for i in range(3):
    for j in range(3):
        eqset1.append(Equations[i,j])
solutionN = sym.solve((eqset1),(x1,x2,x3,x4,x5,x6,x7,x8,x9))
solutionN

{x1: -26.5411295543463,
 x2: 5.39871176886447,
 x3: -21.5208067502588,
 x4: -3.58354638293741,
 x5: 39.7672930341960,
 x6: -61.0818432024975,
 x7: -3.35951240644942,
 x8: -1.64686222935135,
 x9: 69.3446645555354}